In [2]:
from datetime import timedelta, datetime
import rsa_headers as rh
import rsa_data as rd
import main
import config
files = r"C:\Users\MB2705851\Desktop\SYNTELL TEST\0006_20211231.RSA"
import pandas as pd
pd.set_option("display.max_columns", None)

testfile = r"C:\Users\MB2705851\Desktop\Temp\SYNTELL-test.csv"

In [3]:
def get_direction(lane_number, df: pd.DataFrame) -> pd.DataFrame:
    filt = df[1] == lane_number
    df = df.where(filt)
    df = df[2].dropna()
    df = int(df)
    return df

In [4]:
def dtype21(df: pd.DataFrame) -> pd.DataFrame:
    data = df.loc[(df[0] == "21") & (df[1].isin(["0", "1", "2", "3", "4"]))].dropna(
        axis=1, how="all"
    )
    dfh2 = pd.DataFrame(df.loc[(df[0].isin(["S0", "L1"]))]).dropna(
        axis=1, how="all"
    )

    if data.empty:
        pass
    else:
        if (data[1] == "0").any():
            ddf = data.iloc[:, 2:]
            ddf = pd.DataFrame(ddf).dropna(axis=1, how="all")
            ddf.columns = [
                "end_datetime",
                "end_time",
                "duration_min",
                "lane_number",
                "speedbin1",
                "speedbin2",
                "speedbin3",
                "speedbin4",
                "speedbin5",
                "speedbin6",
                "speedbin7",
                "speedbin8",
                "speedbin9",
                "speedbin10",
                "sum_of_heavy_vehicle_speeds",
                "short_heavy_vehicles",
                "medium_heavy_vehicles",
                "long_heavy_vehicles",
                "rear_to_rear_headway_shorter_than_2_seconds",
                "rear_to_rear_headways_shorter_than_programmed_time",
            ]
            ddf["speedbin0"] = 0

        elif (data[1] == "1").any():
            ddf = data.iloc[:, 3:]
            ddf = pd.DataFrame(ddf).dropna(axis=1, how="all")
            ddf.columns = [
                "end_datetime",
                "end_time",
                "duration_min",
                "lane_number",
                "speedbin0",
                "speedbin1",
                "speedbin2",
                "speedbin3",
                "speedbin4",
                "speedbin5",
                "speedbin6",
                "speedbin7",
                "speedbin8",
                "speedbin9",
                "speedbin10",
                "sum_of_heavy_vehicle_speeds",
                "short_heavy_vehicles",
                "medium_heavy_vehicles",
                "long_heavy_vehicles",
                "rear_to_rear_headway_shorter_than_2_seconds",
                "rear_to_rear_headways_shorter_than_programmed_time",
            ]

        ddf = ddf.fillna(0)
        ddf["duration_min"] = ddf["duration_min"].astype(int)
        ddf["lane_number"] = ddf["lane_number"].astype(int)
        ddf["speedbin0"] = ddf["speedbin0"].astype(int)
        ddf["speedbin1"] = ddf["speedbin1"].astype(int)
        ddf["speedbin2"] = ddf["speedbin2"].astype(int)
        ddf["speedbin3"] = ddf["speedbin3"].astype(int)
        ddf["speedbin4"] = ddf["speedbin4"].astype(int)
        ddf["speedbin5"] = ddf["speedbin5"].astype(int)
        ddf["speedbin6"] = ddf["speedbin6"].astype(int)
        ddf["speedbin7"] = ddf["speedbin7"].astype(int)
        ddf["speedbin8"] = ddf["speedbin8"].astype(int)
        ddf["speedbin9"] = ddf["speedbin9"].astype(int)
        ddf["speedbin10"] = ddf["speedbin10"].astype(int)
        ddf["sum_of_heavy_vehicle_speeds"] = ddf[
            "sum_of_heavy_vehicle_speeds"
        ].astype(int)
        ddf["short_heavy_vehicles"] = ddf["short_heavy_vehicles"].astype(int)
        ddf["medium_heavy_vehicles"] = ddf["medium_heavy_vehicles"].astype(int)
        ddf["long_heavy_vehicles"] = ddf["long_heavy_vehicles"].astype(int)
        ddf["rear_to_rear_headway_shorter_than_2_seconds"] = ddf[
            "rear_to_rear_headway_shorter_than_2_seconds"
        ].astype(int)
        ddf["rear_to_rear_headways_shorter_than_programmed_time"] = ddf[
            "rear_to_rear_headways_shorter_than_programmed_time"
        ].astype(int)

        ddf["total_heavy_vehicles_type21"] = (
            ddf["short_heavy_vehicles"]
            + ddf["medium_heavy_vehicles"]
            + ddf["long_heavy_vehicles"]
        )
        ddf["total_heavy_vehicles_type21"] = ddf[
            "total_heavy_vehicles_type21"
        ].astype(int)

        ddf["total_light_vehicles_type21"] = (
            ddf["speedbin1"]
            + ddf["speedbin2"]
            + ddf["speedbin3"]
            + ddf["speedbin4"]
            + ddf["speedbin5"]
            + ddf["speedbin6"]
            + ddf["speedbin7"]
            + ddf["speedbin8"]
            + ddf["speedbin9"]
            + ddf["speedbin10"]
            - ddf["short_heavy_vehicles"]
            - ddf["medium_heavy_vehicles"]
            - ddf["long_heavy_vehicles"]
        )
        ddf["total_light_vehicles_type21"] = ddf[
            "total_light_vehicles_type21"
        ].astype(int)

        ddf["total_vehicles_type21"] = (
            ddf["speedbin1"]
            + ddf["speedbin2"]
            + ddf["speedbin3"]
            + ddf["speedbin4"]
            + ddf["speedbin5"]
            + ddf["speedbin6"]
            + ddf["speedbin7"]
            + ddf["speedbin8"]
            + ddf["speedbin9"]
            + ddf["speedbin10"]
        )
        ddf["total_vehicles_type21"] = ddf["total_vehicles_type21"].astype(int)

        max_lanes = ddf["lane_number"].max()
        ddf["direction"] = ddf.apply(
            lambda x: "P" if x["lane_number"] <= (int(max_lanes) / 2) else "N",
            axis=1,
        )

        direction = dfh2.loc[dfh2[0] == "L1", 1:3].astype(int)
        direction = direction.drop_duplicates()
        try:
            ddf["forward_direction_code"] = ddf.apply(
                lambda x: Data.get_direction(x["lane_number"], direction), axis=1
            )
            # FIXME: ddf['lane_position_code']=ddf.apply(lambda x: Data.get_lane_position(x['lane_number'],direction),axis=1)
        except Exception:
            ddf["forward_direction_code"] = None
            # ddf['lane_position_code']=None

        ddf["end_datetime"] = ddf.apply(
            lambda x: pd.to_datetime(
                x["end_datetime"] + x["end_time"], format="%y%m%d%H%M"
            )
            if (
                x["end_time"] != "2400"
                and len(str(x["end_datetime"])) == 6
                and len(x["end_time"]) == 4
            )
            else (
                pd.to_datetime(
                    x["end_datetime"] + x["end_time"], format="%Y%m%d%H%M"
                )
                if (
                    x["end_time"] != "2400"
                    and len(str(x["end_datetime"])) == 8
                    and len(x["end_time"]) == 4
                )
                else (
                    pd.to_datetime(x["end_datetime"], format="%y%m%d")
                    + timedelta(days=1)
                    if (
                        x["end_time"] == "2400"
                        and len(str(x["end_datetime"])) == 6
                        and len(x["end_time"]) == 4
                    )
                    else (
                        pd.to_datetime(x["end_datetime"], format="%Y%m%d")
                        + timedelta(days=1)
                        if (
                            x["end_time"] != "2400"
                            and len(str(x["end_datetime"])) == 8
                            and len(x["end_time"]) == 4
                        )
                        else pd.to_datetime(x["end_datetime"])
                    )
                )
            ),
            axis=1,
        )
        ddf = ddf.drop(["end_time"], axis=1)
        ddf["start_datetime"] = pd.to_datetime(
            ddf["end_datetime"]
        ) - pd.to_timedelta(ddf["duration_min"].astype(int), unit="m")
        # ddf['year'] = ddf['start_datetime'].dt.year
        t1 = dfh2.loc[dfh2[0] == "S0", 1].unique()
        ddf["site_id"] = str(t1[0])

        ddf = ddf.drop_duplicates()
        ddf["start_datetime"] = ddf["start_datetime"].astype("datetime64[ns]")

        return ddf

In [5]:
df = main.to_df(files)

In [7]:
header = rh.Headers(df)
header = header.header

In [9]:
print("")

In [13]:
data = rd.Data.dtype60(df)

In [15]:
data = df.loc[(df[0] == "30") & (df[1].isin(["0", "1", "2", "3", "4"]))].dropna(
    axis=1, how="all"
)

In [16]:
print(data)

        0  1       2     3   4   5     6     7     8
19     30  0  211201  0015  15  01  0000  0011  0000
20     30  0  211201  0015  15  02  0000  0015  0000
21     30  0  211201  0015  15  03  0000  0009  0000
22     30  0  211201  0015  15  04  0000  0014  0000
31     30  0  211201  0030  15  01  0000  0007  0002
...    .. ..     ...   ...  ..  ..   ...   ...   ...
35710  30  0  211231  2345  15  04  0000  0029  0001
35719  30  0  211231  2400  15  01  0000  0015  0000
35720  30  0  211231  2400  15  02  0000  0027  0000
35721  30  0  211231  2400  15  03  0000  0021  0000
35722  30  0  211231  2400  15  04  0000  0016  0000

[11904 rows x 9 columns]


In [ ]:
dfh2 = pd.DataFrame(df.loc[(df[0].isin(["S0", "L1"]))]).dropna(
    axis=1, how="all"
)

In [21]:
if data.empty:
    pass
else:
    if data[1].all() == "1":
        ddf = data.iloc[:, 3:]
        ddf = pd.DataFrame(ddf).dropna(axis=1, how="all")
        ddf.columns = [
            "end_datetime",
            "end_time",
            "duration_min",
            "lane_number",
            "unknown_vehicle_error_class",
            "motorcycle",
            "light_motor_vehicles",
            "light_motor_vehicles_towing",
            "two_axle_busses",
            "two_axle_6_tyre_single_units",
            "busses_with_3_or_4_axles",
            "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max",
            "three_axle_single_unit_including_single_axle_light_trailer",
            "four_or_less_axle_including_a_single_trailer",
            "buses_with_5_or_more_axles",
            "three_axle_single_unit_and_light_trailer_more_than_4_axles",
            "five_axle_single_trailer",
            "six_axle_single_trailer",
            "five_or_less_axle_multi_trailer",
            "six_axle_multi_trailer",
            "seven_axle_multi_trailer",
            "eight_or_more_axle_multi_trailer",
        ]
        ddf["heavy_vehicle"] = 0

    elif data[1].isin(["0", "2"]).any():
        ddf = data.iloc[:, 2:]
        ddf = pd.DataFrame(ddf).dropna(axis=1, how="all")
        if ddf.shape[1] == 7:
            ddf.columns = [
                "end_datetime",
                "end_time",
                "duration_min",
                "lane_number",
                "unknown_vehicle_error_class",
                "light_motor_vehicles",
                "heavy_vehicle",
            ]
            ddf = pd.concat(
                [
                    ddf,
                    pd.DataFrame(
                        columns=[
                            "motorcycle",
                            "light_motor_vehicles",
                            "light_motor_vehicles_towing",
                            "two_axle_busses",
                            "two_axle_6_tyre_single_units",
                            "busses_with_3_or_4_axles",
                            "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max",
                            "three_axle_single_unit_including_single_axle_light_trailer",
                            "four_or_less_axle_including_a_single_trailer",
                            "buses_with_5_or_more_axles",
                            "three_axle_single_unit_and_light_trailer_more_than_4_axles",
                            "five_axle_single_trailer",
                            "six_axle_single_trailer",
                            "five_or_less_axle_multi_trailer",
                            "six_axle_multi_trailer",
                            "seven_axle_multi_trailer",
                            "eight_or_more_axle_multi_trailer",
                        ]
                    ),
                ]
            )
        else:
            ddf.columns = [
                "end_datetime",
                "end_time",
                "duration_min",
                "lane_number",
                "unknown_vehicle_error_class",
                "motorcycle",
                "light_motor_vehicles",
                "light_motor_vehicles_towing",
                "two_axle_busses",
                "two_axle_6_tyre_single_units",
                "busses_with_3_or_4_axles",
                "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max",
                "three_axle_single_unit_including_single_axle_light_trailer",
                "four_or_less_axle_including_a_single_trailer",
                "buses_with_5_or_more_axles",
                "three_axle_single_unit_and_light_trailer_more_than_4_axles",
                "five_axle_single_trailer",
                "six_axle_single_trailer",
                "five_or_less_axle_multi_trailer",
                "six_axle_multi_trailer",
                "seven_axle_multi_trailer",
                "eight_or_more_axle_multi_trailer",
            ]

    ddf = ddf.dropna(
        axis=0,
        how="all",
        subset=[
            "unknown_vehicle_error_class",
            "light_motor_vehicles",
            "motorcycle",
            "light_motor_vehicles",
            "light_motor_vehicles_towing",
            "two_axle_busses",
            "two_axle_6_tyre_single_units",
            "busses_with_3_or_4_axles",
            "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max",
            "three_axle_single_unit_including_single_axle_light_trailer",
            "four_or_less_axle_including_a_single_trailer",
            "buses_with_5_or_more_axles",
            "three_axle_single_unit_and_light_trailer_more_than_4_axles",
            "five_axle_single_trailer",
            "six_axle_single_trailer",
            "five_or_less_axle_multi_trailer",
            "six_axle_multi_trailer",
            "seven_axle_multi_trailer",
            "eight_or_more_axle_multi_trailer",
        ],
    )

    ddf=ddf.fillna(0)
    ddf["duration_min"] = ddf["duration_min"].astype(int)
    ddf["lane_number"] = ddf["lane_number"].astype(int)
    ddf["unknown_vehicle_error_class"] = ddf[
        "unknown_vehicle_error_class"
    ].astype(int)
    ddf["light_motor_vehicles"] = ddf["light_motor_vehicles"].astype(int)
    ddf["heavy_vehicle"] = ddf["heavy_vehicle"].astype(int)
    ddf["light_motor_vehicles_towing"] = ddf[
        "light_motor_vehicles_towing"
    ].astype(int)
    ddf["two_axle_busses"] = ddf["two_axle_busses"].astype(int)
    ddf["two_axle_6_tyre_single_units"] = ddf[
        "two_axle_6_tyre_single_units"
    ].astype(int)
    ddf["busses_with_3_or_4_axles"] = ddf["busses_with_3_or_4_axles"].astype(
        int
    )
    ddf["two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max"] = ddf[
        "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max"
    ].astype(int)
    ddf["three_axle_single_unit_including_single_axle_light_trailer"] = ddf[
        "three_axle_single_unit_including_single_axle_light_trailer"
    ].astype(int)
    ddf["four_or_less_axle_including_a_single_trailer"] = ddf[
        "four_or_less_axle_including_a_single_trailer"
    ].astype(int)
    ddf["buses_with_5_or_more_axles"] = ddf[
        "buses_with_5_or_more_axles"
    ].astype(int)
    ddf["three_axle_single_unit_and_light_trailer_more_than_4_axles"] = ddf[
        "three_axle_single_unit_and_light_trailer_more_than_4_axles"
    ].astype(int)
    ddf["five_axle_single_trailer"] = ddf["five_axle_single_trailer"].astype(
        int
    )
    ddf["six_axle_single_trailer"] = ddf["six_axle_single_trailer"].astype(int)
    ddf["five_or_less_axle_multi_trailer"] = ddf[
        "five_or_less_axle_multi_trailer"
    ].astype(int)
    ddf["six_axle_multi_trailer"] = ddf["six_axle_multi_trailer"].astype(int)
    ddf["seven_axle_multi_trailer"] = ddf["seven_axle_multi_trailer"].astype(
        int
    )
    ddf["eight_or_more_axle_multi_trailer"] = ddf[
        "eight_or_more_axle_multi_trailer"
    ].astype(int)

    ddf["total_light_vehicles_type30"] = (
        ddf["motorcycle"].astype(int)
        + ddf["light_motor_vehicles"].astype(int)
        + ddf["light_motor_vehicles_towing"].astype(int)
    )
    ddf["total_heavy_vehicles_type30"] = (
        ddf["two_axle_busses"].astype(int)
        + ddf["two_axle_6_tyre_single_units"].astype(int)
        + ddf["busses_with_3_or_4_axles"].astype(int)
        + ddf[
            "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max"
        ].astype(int)
        + ddf[
            "three_axle_single_unit_including_single_axle_light_trailer"
        ].astype(int)
        + ddf["four_or_less_axle_including_a_single_trailer"].astype(int)
        + ddf["buses_with_5_or_more_axles"].astype(int)
        + ddf[
            "three_axle_single_unit_and_light_trailer_more_than_4_axles"
        ].astype(int)
        + ddf["five_axle_single_trailer"].astype(int)
        + ddf["six_axle_single_trailer"].astype(int)
        + ddf["five_or_less_axle_multi_trailer"].astype(int)
        + ddf["six_axle_multi_trailer"].astype(int)
        + ddf["seven_axle_multi_trailer"].astype(int)
        + ddf["eight_or_more_axle_multi_trailer"].astype(int)
    )

    ddf["total_vehicles_type30"] = (
        ddf["motorcycle"].astype(int)
        + ddf["light_motor_vehicles"].astype(int)
        + ddf["light_motor_vehicles_towing"].astype(int)
        + ddf["two_axle_busses"].astype(int)
        + ddf["two_axle_6_tyre_single_units"].astype(int)
        + ddf["busses_with_3_or_4_axles"].astype(int)
        + ddf[
            "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max"
        ].astype(int)
        + ddf[
            "three_axle_single_unit_including_single_axle_light_trailer"
        ].astype(int)
        + ddf["four_or_less_axle_including_a_single_trailer"].astype(int)
        + ddf["buses_with_5_or_more_axles"].astype(int)
        + ddf[
            "three_axle_single_unit_and_light_trailer_more_than_4_axles"
        ].astype(int)
        + ddf["five_axle_single_trailer"].astype(int)
        + ddf["six_axle_single_trailer"].astype(int)
        + ddf["five_or_less_axle_multi_trailer"].astype(int)
        + ddf["six_axle_multi_trailer"].astype(int)
        + ddf["seven_axle_multi_trailer"].astype(int)
        + ddf["eight_or_more_axle_multi_trailer"].astype(int)
    )

    max_lanes = ddf["lane_number"].max()
    ddf["direction"] = ddf.apply(
        lambda x: "P" if x["lane_number"] <= (int(max_lanes) / 2) else "N",
        axis=1,
    )
    # direction = dfh2.loc[dfh2[0] == "L1", 1:3].astype(int)
    # direction = direction.drop_duplicates()
    try:
        ddf["forward_direction_code"] = ddf.apply(
            lambda x: Data.get_direction(x["lane_number"], direction), axis=1
        )
        # FIXME: ddf['lane_position_code']=ddf.apply(lambda x: Data.get_lane_position(x['lane_number'],direction),axis=1)
    except Exception:
        ddf["forward_direction_code"] = None
        # ddf['lane_position_code']=None

    ddf["end_datetime"] = ddf.apply(
        lambda x: pd.to_datetime(
            x["end_datetime"] + x["end_time"], format="%y%m%d%H%M"
        )
        if (
            x["end_time"] != "2400"
            and len(str(x["end_datetime"])) == 6
            and len(x["end_time"]) == 4
        )
        else (
            pd.to_datetime(
                x["end_datetime"] + x["end_time"], format="%Y%m%d%H%M"
            )
            if (
                x["end_time"] != "2400"
                and len(str(x["end_datetime"])) == 8
                and len(x["end_time"]) == 4
            )
            else (
                pd.to_datetime(x["end_datetime"], format="%y%m%d")
                + timedelta(days=1)
                if (
                    x["end_time"] == "2400"
                    and len(str(x["end_datetime"])) == 6
                    and len(x["end_time"]) == 4
                )
                else (
                    pd.to_datetime(x["end_datetime"], format="%Y%m%d")
                    + timedelta(days=1)
                    if (
                        x["end_time"] != "2400"
                        and len(str(x["end_datetime"])) == 8
                        and len(x["end_time"]) == 4
                    )
                    else pd.to_datetime(x["end_datetime"])
                )
            )
        ),
        axis=1,
    )

    ddf = ddf.drop(["end_time"], axis=1)

    ddf["start_datetime"] = pd.to_datetime(
        ddf["end_datetime"]
    ) - pd.to_timedelta(ddf["duration_min"].astype(int), unit="m")
    # ddf['year'] = ddf['start_datetime'].dt.year
    # t1 = dfh2.loc[dfh2[0] == "S0", 1].unique()
    # ddf["site_id"] = str(t1[0])

    ddf = ddf.drop_duplicates()
    ddf["start_datetime"] = ddf["start_datetime"].astype("datetime64[ns]")

    print(ddf)

             end_datetime  duration_min  lane_number  \
19    2021-12-01 00:15:00            15            1   
20    2021-12-01 00:15:00            15            2   
21    2021-12-01 00:15:00            15            3   
22    2021-12-01 00:15:00            15            4   
31    2021-12-01 00:30:00            15            1   
...                   ...           ...          ...   
35710 2021-12-31 23:45:00            15            4   
35719 2022-01-01 00:00:00            15            1   
35720 2022-01-01 00:00:00            15            2   
35721 2022-01-01 00:00:00            15            3   
35722 2022-01-01 00:00:00            15            4   

       unknown_vehicle_error_class  light_motor_vehicles  heavy_vehicle  \
19                               0                    11              0   
20                               0                    15              0   
21                               0                     9              0   
22                         

In [ ]:
data = df.loc[(df[0] == "30") & (df[1].isin(["0", "1", "2", "3", "4"]))].dropna(
    axis=1, how="all"
)
dfh2 = pd.DataFrame(df.loc[(df[0].isin(["S0", "L1"]))]).dropna(
    axis=1, how="all"
)

In [ ]:
def dtype30(df: pd.DataFrame) -> pd.DataFrame:
    data = df.loc[(df[0] == "30") & (df[1].isin(["0", "1", "2", "3", "4"]))].dropna(
        axis=1, how="all"
    )
    dfh2 = pd.DataFrame(df.loc[(df[0].isin(["S0", "L1"]))]).dropna(
        axis=1, how="all"
    )
    if data.empty:
        pass
    else:
        if data[1].all() == "1":
            ddf = data.iloc[:, 3:]
            ddf = pd.DataFrame(ddf).dropna(axis=1, how="all")
            ddf.columns = [
                "end_datetime",
                "end_time",
                "duration_min",
                "lane_number",
                "unknown_vehicle_error_class",
                "motorcycle",
                "light_motor_vehicles",
                "light_motor_vehicles_towing",
                "two_axle_busses",
                "two_axle_6_tyre_single_units",
                "busses_with_3_or_4_axles",
                "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max",
                "three_axle_single_unit_including_single_axle_light_trailer",
                "four_or_less_axle_including_a_single_trailer",
                "buses_with_5_or_more_axles",
                "three_axle_single_unit_and_light_trailer_more_than_4_axles",
                "five_axle_single_trailer",
                "six_axle_single_trailer",
                "five_or_less_axle_multi_trailer",
                "six_axle_multi_trailer",
                "seven_axle_multi_trailer",
                "eight_or_more_axle_multi_trailer",
            ]
            ddf["heavy_vehicle"] = 0

        elif data[1].isin(["0","2"]).any():
            ddf = data.iloc[:, 2:]
            ddf = pd.DataFrame(ddf).dropna(axis=1, how="all")
            if ddf.shape[1] == 7:
                ddf.columns = [
                    "end_datetime",
                    "end_time",
                    "duration_min",
                    "lane_number",
                    "unknown_vehicle_error_class",
                    "light_motor_vehicles",
                    "heavy_vehicle",
                ]
                ddf = pd.concat(
                    [
                        ddf,
                        pd.DataFrame(
                            columns=[
                                "motorcycle",
                                "light_motor_vehicles",
                                "light_motor_vehicles_towing",
                                "two_axle_busses",
                                "two_axle_6_tyre_single_units",
                                "busses_with_3_or_4_axles",
                                "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max",
                                "three_axle_single_unit_including_single_axle_light_trailer",
                                "four_or_less_axle_including_a_single_trailer",
                                "buses_with_5_or_more_axles",
                                "three_axle_single_unit_and_light_trailer_more_than_4_axles",
                                "five_axle_single_trailer",
                                "six_axle_single_trailer",
                                "five_or_less_axle_multi_trailer",
                                "six_axle_multi_trailer",
                                "seven_axle_multi_trailer",
                                "eight_or_more_axle_multi_trailer",
                            ]
                        ),
                    ]
                )
            else:
                ddf.columns = [
                    "end_datetime",
                    "end_time",
                    "duration_min",
                    "lane_number",
                    "unknown_vehicle_error_class",
                    "motorcycle",
                    "light_motor_vehicles",
                    "light_motor_vehicles_towing",
                    "two_axle_busses",
                    "two_axle_6_tyre_single_units",
                    "busses_with_3_or_4_axles",
                    "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max",
                    "three_axle_single_unit_including_single_axle_light_trailer",
                    "four_or_less_axle_including_a_single_trailer",
                    "buses_with_5_or_more_axles",
                    "three_axle_single_unit_and_light_trailer_more_than_4_axles",
                    "five_axle_single_trailer",
                    "six_axle_single_trailer",
                    "five_or_less_axle_multi_trailer",
                    "six_axle_multi_trailer",
                    "seven_axle_multi_trailer",
                    "eight_or_more_axle_multi_trailer",
                ]

        ddf = ddf.dropna(
            axis=0,
            how="all",
            subset=[
                "unknown_vehicle_error_class",
                "light_motor_vehicles",
                "motorcycle",
                "light_motor_vehicles",
                "light_motor_vehicles_towing",
                "two_axle_busses",
                "two_axle_6_tyre_single_units",
                "busses_with_3_or_4_axles",
                "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max",
                "three_axle_single_unit_including_single_axle_light_trailer",
                "four_or_less_axle_including_a_single_trailer",
                "buses_with_5_or_more_axles",
                "three_axle_single_unit_and_light_trailer_more_than_4_axles",
                "five_axle_single_trailer",
                "six_axle_single_trailer",
                "five_or_less_axle_multi_trailer",
                "six_axle_multi_trailer",
                "seven_axle_multi_trailer",
                "eight_or_more_axle_multi_trailer",
            ],
        )

        # ddf=ddf.fillna(0)
        ddf["duration_min"] = ddf["duration_min"].astype(int)
        ddf["lane_number"] = ddf["lane_number"].astype(int)
        ddf["unknown_vehicle_error_class"] = ddf[
            "unknown_vehicle_error_class"
        ].astype(int)
        ddf["light_motor_vehicles"] = ddf["light_motor_vehicles"].astype(int)
        ddf["light_motor_vehicles_towing"] = ddf[
            "light_motor_vehicles_towing"
        ].astype(int)
        ddf["two_axle_busses"] = ddf["two_axle_busses"].astype(int)
        ddf["two_axle_6_tyre_single_units"] = ddf[
            "two_axle_6_tyre_single_units"
        ].astype(int)
        ddf["busses_with_3_or_4_axles"] = ddf["busses_with_3_or_4_axles"].astype(
            int
        )
        ddf["two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max"] = ddf[
            "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max"
        ].astype(int)
        ddf["three_axle_single_unit_including_single_axle_light_trailer"] = ddf[
            "three_axle_single_unit_including_single_axle_light_trailer"
        ].astype(int)
        ddf["four_or_less_axle_including_a_single_trailer"] = ddf[
            "four_or_less_axle_including_a_single_trailer"
        ].astype(int)
        ddf["buses_with_5_or_more_axles"] = ddf[
            "buses_with_5_or_more_axles"
        ].astype(int)
        ddf["three_axle_single_unit_and_light_trailer_more_than_4_axles"] = ddf[
            "three_axle_single_unit_and_light_trailer_more_than_4_axles"
        ].astype(int)
        ddf["five_axle_single_trailer"] = ddf["five_axle_single_trailer"].astype(
            int
        )
        ddf["six_axle_single_trailer"] = ddf["six_axle_single_trailer"].astype(int)
        ddf["five_or_less_axle_multi_trailer"] = ddf[
            "five_or_less_axle_multi_trailer"
        ].astype(int)
        ddf["six_axle_multi_trailer"] = ddf["six_axle_multi_trailer"].astype(int)
        ddf["seven_axle_multi_trailer"] = ddf["seven_axle_multi_trailer"].astype(
            int
        )
        ddf["eight_or_more_axle_multi_trailer"] = ddf[
            "eight_or_more_axle_multi_trailer"
        ].astype(int)

        ddf["total_light_vehicles_type30"] = (
            ddf["motorcycle"].astype(int)
            + ddf["light_motor_vehicles"].astype(int)
            + ddf["light_motor_vehicles_towing"].astype(int)
        )
        ddf["total_heavy_vehicles_type30"] = (
            ddf["two_axle_busses"].astype(int)
            + ddf["two_axle_6_tyre_single_units"].astype(int)
            + ddf["busses_with_3_or_4_axles"].astype(int)
            + ddf[
                "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max"
            ].astype(int)
            + ddf[
                "three_axle_single_unit_including_single_axle_light_trailer"
            ].astype(int)
            + ddf["four_or_less_axle_including_a_single_trailer"].astype(int)
            + ddf["buses_with_5_or_more_axles"].astype(int)
            + ddf[
                "three_axle_single_unit_and_light_trailer_more_than_4_axles"
            ].astype(int)
            + ddf["five_axle_single_trailer"].astype(int)
            + ddf["six_axle_single_trailer"].astype(int)
            + ddf["five_or_less_axle_multi_trailer"].astype(int)
            + ddf["six_axle_multi_trailer"].astype(int)
            + ddf["seven_axle_multi_trailer"].astype(int)
            + ddf["eight_or_more_axle_multi_trailer"].astype(int)
        )

        ddf["total_vehicles_type30"] = (
            ddf["motorcycle"].astype(int)
            + ddf["light_motor_vehicles"].astype(int)
            + ddf["light_motor_vehicles_towing"].astype(int)
            + ddf["two_axle_busses"].astype(int)
            + ddf["two_axle_6_tyre_single_units"].astype(int)
            + ddf["busses_with_3_or_4_axles"].astype(int)
            + ddf[
                "two_axle_6_tyre_single_unit_with_light_trailer_4_axles_max"
            ].astype(int)
            + ddf[
                "three_axle_single_unit_including_single_axle_light_trailer"
            ].astype(int)
            + ddf["four_or_less_axle_including_a_single_trailer"].astype(int)
            + ddf["buses_with_5_or_more_axles"].astype(int)
            + ddf[
                "three_axle_single_unit_and_light_trailer_more_than_4_axles"
            ].astype(int)
            + ddf["five_axle_single_trailer"].astype(int)
            + ddf["six_axle_single_trailer"].astype(int)
            + ddf["five_or_less_axle_multi_trailer"].astype(int)
            + ddf["six_axle_multi_trailer"].astype(int)
            + ddf["seven_axle_multi_trailer"].astype(int)
            + ddf["eight_or_more_axle_multi_trailer"].astype(int)
        )

        max_lanes = ddf["lane_number"].max()
        ddf["direction"] = ddf.apply(
            lambda x: "P" if x["lane_number"] <= (int(max_lanes) / 2) else "N",
            axis=1,
        )
        direction = dfh2.loc[dfh2[0] == "L1", 1:3].astype(int)
        direction = direction.drop_duplicates()
        try:
            ddf["forward_direction_code"] = ddf.apply(
                lambda x: Data.get_direction(x["lane_number"], direction), axis=1
            )
            # FIXME: ddf['lane_position_code']=ddf.apply(lambda x: Data.get_lane_position(x['lane_number'],direction),axis=1)
        except Exception:
            ddf["forward_direction_code"] = None
            # ddf['lane_position_code']=None

        ddf["end_datetime"] = ddf.apply(
            lambda x: pd.to_datetime(
                x["end_datetime"] + x["end_time"], format="%y%m%d%H%M"
            )
            if (
                x["end_time"] != "2400"
                and len(str(x["end_datetime"])) == 6
                and len(x["end_time"]) == 4
            )
            else (
                pd.to_datetime(
                    x["end_datetime"] + x["end_time"], format="%Y%m%d%H%M"
                )
                if (
                    x["end_time"] != "2400"
                    and len(str(x["end_datetime"])) == 8
                    and len(x["end_time"]) == 4
                )
                else (
                    pd.to_datetime(x["end_datetime"], format="%y%m%d")
                    + timedelta(days=1)
                    if (
                        x["end_time"] == "2400"
                        and len(str(x["end_datetime"])) == 6
                        and len(x["end_time"]) == 4
                    )
                    else (
                        pd.to_datetime(x["end_datetime"], format="%Y%m%d")
                        + timedelta(days=1)
                        if (
                            x["end_time"] != "2400"
                            and len(str(x["end_datetime"])) == 8
                            and len(x["end_time"]) == 4
                        )
                        else pd.to_datetime(x["end_datetime"])
                    )
                )
            ),
            axis=1,
        )

        ddf = ddf.drop(["end_time"], axis=1)

        ddf["start_datetime"] = pd.to_datetime(
            ddf["end_datetime"]
        ) - pd.to_timedelta(ddf["duration_min"].astype(int), unit="m")
        # ddf['year'] = ddf['start_datetime'].dt.year
        t1 = dfh2.loc[dfh2[0] == "S0", 1].unique()
        ddf["site_id"] = str(t1[0])

        ddf = ddf.drop_duplicates()

        return ddf

In [ ]:
DATA = rd.Data(df)

In [20]:
data = DATA.dtype21
data = main.data_join(data, header)
data.drop("station_name", axis=1, inplace=True)

d2 = DATA.dtype30
if d2 is None:
    pass
else:
    d2 = main.data_join(d2, header)
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )

d2 = DATA.dtype70
if d2 is None:
    pass
else:
    data = main.data_join(d2, header)
    data.drop("station_name", axis=1, inplace=True)
    data['start_datetime'] = data['start_datetime'].astype('datetime64[ns]')
    d2['start_datetime'] = d2['start_datetime'].astype('datetime64[ns]')
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )


d2 = DATA.dtype10
if d2 is None:
    pass
else:
    data = main.data_join(d2, header)
    data.drop("station_name", axis=1, inplace=True)
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )


d2 = DATA.dtype60
if d2 is None:
    pass
else:
    data = main.data_join(d2, header)
    data.drop("station_name", axis=1, inplace=True)
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )


In [14]:
# data = data.rename(columns = lambda x: x[:-2] if re.search(r'$\_x', x),)
# data = data.rename(columns=(lambda x: x[:-2] if '_x' in x else x))
# data.columns.str.replace(r'\_x$', '', regex=True)
print("")

In [23]:
data = data[data.columns.intersection(config.DATA_COLUMN_NAMES)]
header = header[header.columns.intersection(config.HEADER_COLUMN_NAMES)]

print(data)

                                 header_id                end_datetime  \
0     32b5dfb9-3194-4bb8-9e21-0cc563dbab68  2021-10-29 13:00:00.000000   
1     32b5dfb9-3194-4bb8-9e21-0cc563dbab68  2021-10-29 13:00:00.000000   
2     32b5dfb9-3194-4bb8-9e21-0cc563dbab68  2021-10-29 13:15:00.000000   
3     32b5dfb9-3194-4bb8-9e21-0cc563dbab68  2021-10-29 13:15:00.000000   
4     32b5dfb9-3194-4bb8-9e21-0cc563dbab68  2021-10-29 13:30:00.000000   
...                                    ...                         ...   
6231  7842979d-22d8-406e-b2f3-c0580443e0af  2021-12-01 00:00:00.000000   
6232                                   NaN                         NaN   
6233                                   NaN                         NaN   
6234                                   NaN                         NaN   
6235                                   NaN                         NaN   

      duration_min  lane_number  total_free_flowing_light_vehicles  \
0             15.0            1          

In [ ]:
data = DATA.dtype21
data = main.data_join(data, header)
data.drop("station_name", axis=1, inplace=True)

d2 = DATA.dtype30
if d2 is None:
    pass
else:
    d2 = main.data_join(d2, header)
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )

d2 = DATA.dtype70
if d2 is None:
    pass
else:
    data = main.data_join(d2, header)
    data.drop("station_name", axis=1, inplace=True)
    data['start_datetime'] = data['start_datetime'].astype('datetime64[ns]')
    d2['start_datetime'] = d2['start_datetime'].astype('datetime64[ns]')
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )

d2 = DATA.dtype10
if d2 is None:
    pass
else:
    data = main.data_join(d2, header)
    data.drop("station_name", axis=1, inplace=True)
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )


d2 = DATA.dtype60
if d2 is None:
    pass
else:
    data = main.data_join(d2, header)
    data.drop("station_name", axis=1, inplace=True)
    data = data.merge(
        d2, how="outer", on=["site_id", "start_datetime", "lane_number"]
    )


In [ ]:

data2 = main.data_join(data, header.header)
data.drop('station_name', axis=1, inplace=True)
print(data2)

In [ ]:
print(data)

In [ ]:
data.to_csv(testfile)

In [ ]:
main.push_to_db(
            data,
            "electronic_count_data_partitioned",
            ["site_id", "start_datetime", "lane_number"],
        )